In [2]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import os
import glob
import sys
sys.path.insert(0, '../scripts/')
from football_field import create_football_field
from plots import plot_play
import math

%matplotlib inline
pd.options.display.max_columns = 100

%load_ext autoreload
%autoreload 2

In [6]:
df = pd.read_parquet('../working/max_risk_partners.parquet')
vr = pd.read_csv('../working/video_review-detailed.csv')

In [7]:
injury_risk_factor =pd.merge(vr,
                             df,
                             left_on=['Season_Year','GameKey','PlayID','GSISID','Primary_Partner_GSISID'],
                             right_on=['season_year','gamekey','playid','gsisid','gsisid_partner'],
                             how='left')

In [8]:
risk_metrics_by_rolepair = df.groupby(['role','role_partner'])['risk_factor'].agg([np.mean, np.std, 'count']).reset_index()

In [11]:
risk = pd.merge(df, risk_metrics_by_rolepair, suffixes=('','_risk_factor_pair'))

In [12]:
risk

,season_year,gamekey,playid,time,left_to_right,gsisid,gsisid_partner,role,role_partner,generalized_role,generalized_role_partner,punting_returning_team,punting_returning_team_partner,x,y,x_partner,y_partner,dir,o,dir_partner,o_partner,dist,momentum,momentum_partner,opp_momentum,risk_factor,mean,std,count
0,2016,73,2499,2016-09-11 18:57:54.800,1.0,32214.0,31291.0,PLG,GR,Punting_Lineman,Gunner,Punting_Team,Punting_Team,82.239998,49.820000,81.489998,49.290001,128.119995,37.990002,320.200012,206.679993,0.918367,81.580262,244.740785,324.963354,353.848960,165.281045,518.445177,6194
1,2017,408,200,2017-09-10 17:08:04.300,1.0,28100.0,32007.0,PLG,GR,Punting_Lineman,Gunner,Punting_Team,Punting_Team,33.119999,24.959999,36.740002,9.400000,281.019989,330.209991,94.949997,355.320007,15.975544,611.852000,571.061846,1181.256644,73.941560,165.281045,518.445177,6194
2,2017,563,3081,2017-11-26 20:19:21.200,0.0,32134.0,32543.0,PLG,GR,Punting_Lineman,Gunner,Punting_Team,Punting_Team,23.020000,21.480000,19.180000,24.540001,287.329987,208.029999,174.899994,48.099998,4.910113,489.481570,112.172862,542.286471,110.442768,165.281045,518.445177,6194
3,2016,231,3347,2016-11-27 20:29:46.200,1.0,30780.0,33091.0,PLG,GR,Punting_Lineman,Gunner,Punting_Team,Punting_Team,47.810001,30.920000,51.009998,22.660000,68.349998,337.829987,28.450001,318.429993,8.858193,571.061846,826.000170,533.524702,60.229518,165.281045,518.445177,6194
4,2017,618,2347,2017-12-17 22:52:49.400,1.0,29119.0,33114.0,PLG,GR,Punting_Lineman,Gunner,Punting_Team,Punting_Team,41.919998,30.879999,44.480000,9.660000,285.279999,5.740000,110.449997,5.520000,21.373862,438.493923,438.493923,876.095434,40.989103,165.281045,518.445177,6194
5,2016,51,1368,2016-09-02 01:03:01.900,1.0,32448.0,32849.0,PLG,GR,Punting_Lineman,Gunner,Punting_Team,Punting_Team,85.900002,19.850000,96.269997,15.170000,128.449997,5.940000,5.940000,330.640015,11.377135,224.345723,795.407539,935.315404,82.210100,165.281045,518.445177,6194
6,2016,317,135,2017-01-01 21:28:19.500,1.0,31935.0,31844.0,PLG,GR,Punting_Lineman,Gunner,Punting_Team,Punting_Team,77.820000,34.869999,90.489998,32.970001,84.989998,12.920000,340.100006,225.660004,12.811668,652.642093,622.049523,1010.707465,78.889606,165.281045,518.445177,6194
7,2017,359,3853,2017-08-18 01:44:49.400,1.0,32153.0,29876.0,PLG,GR,Punting_Lineman,Gunner,Punting_Team,Punting_Team,82.500000,45.490002,98.629997,38.090000,72.389999,294.549988,39.599998,281.179993,17.746460,275.333400,989.160723,772.231188,43.514661,165.281045,518.445177,6194
8,2017,565,634,2017-11-26 18:26:12.800,0.0,23259.0,32617.0,PLG,GR,Punting_Lineman,Gunner,Punting_Team,Punting_Team,39.580002,0.710000,37.119999,0.700000,315.739990,259.679993,108.169998,226.869995,2.460023,142.765462,91.777800,228.111412,92.727341,165.281045,518.445177,6194
9,2016,310,2657,2017-01-01 20:03:14.500,0.0,31000.0,32617.0,PLG,GR,Punting_Lineman,Gunner,Punting_Team,Punting_Team,35.810001,39.750000,32.250000,38.080002,299.910004,228.720001,178.410004,35.070000,3.932239,734.222400,91.777800,786.080886,199.906690,165.281045,518.445177,6194
